In [7]:
import cPickle
import itertools

with open("/Users/pushpendrerastogi/export/kbmvlsa/dbpedia.trecweb.make/field_tokens.pkl") as f:
    data = cPickle.load(f)

fields = ["DOCNO", "DOCHDR", "names", "category", "attributes", "SimEn", "RelEn"]
for idx, (vocab, field) in enumerate(itertools.izip(data, fields)):
    data[idx] = sorted(data[idx].iteritems(), key=lambda x: (x[1], x[0]), reverse=True)

In [43]:

sum([len(d) for d in data])

12091886

In [7]:
%%cython --cplus
cdef class A:
    cdef public int x

cdef class Alist:
    def __init__(self):
            self.inner = []
    cdef list inner
    cdef void append(self, A a):
        self.inner.append(a)
    cdef A get(self, int i):
        return <A> self.inner[i]
    def __len__(self):
        return len(self.inner)

cdef Alist make_typed_list(int N):
    cdef A a
    cdef int i
    cdef Alist L = Alist()
    for i in range(N):
        a = A()
        a.x = 1
        L.append(a)
    return L

cdef list make_python_list(int N):
    cdef A a
    cdef int i
    cdef list L = []
    for i in range(N):
        a = A()
        a.x = 1
        L.append(a)
    return L

cdef long test_python_list(list L) except -1:
    cdef int i
    cdef long sum = 0
    for i in range(len(L)):
        sum += L[i].x
    return sum

cdef long test_python_list_casted(list L) except -1:
    cdef int i
    cdef long sum = 0
    for i in range(len(L)):
        sum += (<A>L[i]).x
    return sum


cdef long test_python_list_casted_natural(list L) except -1:
    cdef int i
    cdef long sum = 0
    for a in L:
        sum += (<A>a).x
    return sum

cdef long test_python_list_casted_natural_typechecked(list L) except -1:
    cdef int i
    cdef long sum = 0
    for a in L:
        sum += (<A?>a).x
    return sum

cdef long test_typed_list(Alist L) except -1:
    cdef int i
    cdef long sum = 0
    for i in range(len(L)):
        sum += L.get(i).x
    return sum

from time import time

L = make_python_list(10000000)
start = time(); z = test_python_list(L); end = time(); print "Python list", end - start, "s"
start = time(); z = test_python_list_casted(L); end = time(); print "Python list _casted", end - start, "s"
start = time(); z = test_python_list_casted_natural(L); end = time(); print "Python list _casted_natural", end - start, "s"
start = time(); z = test_python_list_casted_natural_typechecked(L); end = time(); print "Python list _casted_natural_typechecked", end - start, "s"

L = make_typed_list(10000000)
start = time()
z = test_typed_list(L)
end = time()
print "Typed list", end - start, "s"

Python list 0.746551036835 s
Python list _casted 0.147969007492 s
Python list _casted_natural 0.127753019333 s
Python list _casted_natural_typechecked 0.126861095428 s
Typed list 0.173035860062 s


In [38]:
%%cython --cplus
from libcpp.vector cimport vector
from libcpp.string cimport string
from libcpp.stack cimport stack
from libcpp.pair cimport pair
from cython.operator cimport dereference as deref, postincrement


cdef FieldedHitList fhl = FieldedHitList(2, [3, 4])
print fhl.field_token_doc_count
fhl.update(0, 1, 234)
fhl.update(0, 1, 234)
fhl.update(0, 1, 235)
print fhl.field_token_doc_count


[[[], [], []], [[], [], [], []]]
[[[], [(234, 2), (235, 1)], []], [[], [], [], []]]


In [65]:
with open("/tmp/tmp.pkl") as f:
    data2 = cPickle.load(f)

#fields = ["DOCNO", "DOCHDR", "names", "category", "attributes", "SimEn", "RelEn"]
#for idx, (vocab, field) in enumerate(itertools.izip(data2, fields)):
#    data2[idx] = sorted(data2[idx].iteritems(), key=lambda x: (x[1], x[0]), reverse=True)

In [1]:
print 1

1


In [67]:
data2[5]['carmela\x00'] == 2

True